In [90]:
# Import requests library
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [76]:
import json
from pandas.io.json import json_normalize

In [77]:
# Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company. 
# Do it for all the flights between two dates <<that cost the same.>>

In [78]:
'''
200: Everything went okay and the result has been returned (if any).
301: The server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
400: The server thinks you made a bad request. This happens when you don’t send along the right data, among other things.
401: You are not properly authenticated.
403: The resource you’re trying to access is forbidden: you don’t have the right permissions to get it.
404: The resource you tried to access doesn't exist.
503: The server can't handle the request.
'''

"\n200: Everything went okay and the result has been returned (if any).\n301: The server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.\n400: The server thinks you made a bad request. This happens when you don’t send along the right data, among other things.\n401: You are not properly authenticated.\n403: The resource you’re trying to access is forbidden: you don’t have the right permissions to get it.\n404: The resource you tried to access doesn't exist.\n503: The server can't handle the request.\n"

In [10]:
# we do not want to leave an API key in code that gets uploaded to GitHub
# I use getpass (because this is interactive)
# for an automated workflow you could store the key in your environment, and read it from there
import getpass
rapidAPIkey = getpass.getpass()


········


In [79]:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"
    params = {"query": city_name}
    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
               'x-rapidapi-key': rapidAPIkey}
    response = requests.get(url, headers = headers, params = params)
    return response.json()["Places"][0]["PlaceId"]

# # If null values are found:

# def city_code(city_name):
#     url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"

#     params = {"query": city_name}

#     headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
#                       'x-rapidapi-key': rapidAPIkey}

#     response = requests.get(url, headers = headers, params = params)
    
#     if len(response.json()["Places"]) > 0:
#         return response.json()["Places"][0]["PlaceId"]
#     else:
#         return ""

def flight_prices(departure, arrival, date):
    departure_code = city_code(departure)
    arrival_code = city_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date}"
    params = {"inboundpartialdate":{date}}
    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': rapidAPIkey}
    response = requests.get(url, headers=headers, params=params)
    return response.json()


In [80]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format())

In [81]:
get_dates("2021-09-17", "2021-09-18")

0    2021-09-17
1    2021-09-18
dtype: object

In [82]:
def flight_days(origin, destination, start, end):
    dates = get_dates(start, end)
    return {date:flight_prices(origin, destination, date) for date in dates}

In [83]:
overview = flight_days("Amsterdam", "Lanzarote", "2021-09-17", "2021-09-18")

In [84]:
overview

{'2021-09-17': {'Quotes': [{'QuoteId': 1,
    'MinPrice': 121,
    'Direct': False,
    'OutboundLeg': {'CarrierIds': [50441],
     'OriginId': 40595,
     'DestinationId': 40101,
     'DepartureDate': '2021-09-17T00:00:00'},
    'QuoteDateTime': '2021-09-13T20:13:00'}],
  'Carriers': [{'CarrierId': 50441, 'Name': 'easyJet'}],
  'Places': [{'Name': 'Lanzarote',
    'Type': 'Station',
    'PlaceId': 40101,
    'IataCode': 'ACE',
    'SkyscannerCode': 'ACE',
    'CityName': 'Lanzarote',
    'CityId': 'ARRE',
    'CountryName': 'Spain'},
   {'Name': 'Amsterdam',
    'Type': 'Station',
    'PlaceId': 40595,
    'IataCode': 'AMS',
    'SkyscannerCode': 'AMS',
    'CityName': 'Amsterdam',
    'CityId': 'AMST',
    'CountryName': 'Netherlands'}],
  'Currencies': [{'Code': 'USD',
    'Symbol': '$',
    'ThousandsSeparator': ',',
    'DecimalSeparator': '.',
    'SymbolOnLeft': True,
    'SpaceBetweenAmountAndSymbol': False,
    'RoundingCoefficient': 0,
    'DecimalDigits': 2}]},
 '2021-09-18'

In [139]:
new = pd.DataFrame(overview).T.reset_index()
new

,index,Quotes,Carriers,Places,Currencies
0,2021-09-17,"[{'QuoteId': 1, 'MinPrice': 121, 'Direct': Fal...","[{'CarrierId': 50441, 'Name': 'easyJet'}]","[{'Name': 'Lanzarote', 'Type': 'Station', 'Pla...","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."
1,2021-09-18,"[{'QuoteId': 1, 'MinPrice': 122, 'Direct': Fal...","[{'CarrierId': 50441, 'Name': 'easyJet'}, {'Ca...","[{'Name': 'Lanzarote', 'Type': 'Station', 'Pla...","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."


In [196]:
quotes = (new["Quotes"][0])
carriers = (new["Carriers"][0])
places = (new["Places"][0])
currencies = (new["Currencies"][0])

# Name_list = df["Name"].tolist()

In [197]:
quotes1 = (new["Quotes"][1])
carriers1 = (new["Carriers"][1])
places1 = (new["Places"][1])
currencies1 = (new["Currencies"][1])

# Name_list = df["Name"].tolist()

In [198]:
all_quotes = quotes + quotes1
all_carriers = carriers + carriers1
all_places = places + places1
all_currencies = currencies + currencies1

In [203]:
pd.json_normalize(all_quotes)
pd.json_normalize(all_carriers)
pd.json_normalize(all_places)
pd.json_normalize(all_currencies)

# pd.json_normalize(quotes) just returns the first row

,Code,Symbol,ThousandsSeparator,DecimalSeparator,SymbolOnLeft,SpaceBetweenAmountAndSymbol,RoundingCoefficient,DecimalDigits
0,USD,$,",",.,True,False,0,2
1,USD,$,",",.,True,False,0,2


In [204]:
pd.json_normalize(all_quotes)

,QuoteId,MinPrice,Direct,QuoteDateTime,OutboundLeg.CarrierIds,OutboundLeg.OriginId,OutboundLeg.DestinationId,OutboundLeg.DepartureDate
0,1,121,False,2021-09-13T20:13:00,[50441],40595,40101,2021-09-17T00:00:00
1,1,122,False,2021-09-14T14:51:00,[50441],40595,40101,2021-09-18T00:00:00
2,2,322,True,2021-09-14T22:07:00,[1190],40595,40101,2021-09-18T00:00:00


In [205]:
pd.json_normalize(all_carriers)

,CarrierId,Name
0,50441,easyJet
1,50441,easyJet
2,1190,transavia


In [206]:
pd.json_normalize(all_places)

,Name,Type,PlaceId,IataCode,SkyscannerCode,CityName,CityId,CountryName
0,Lanzarote,Station,40101,ACE,ACE,Lanzarote,ARRE,Spain
1,Amsterdam,Station,40595,AMS,AMS,Amsterdam,AMST,Netherlands
2,Lanzarote,Station,40101,ACE,ACE,Lanzarote,ARRE,Spain
3,Amsterdam,Station,40595,AMS,AMS,Amsterdam,AMST,Netherlands


In [207]:
pd.json_normalize(all_currencies)

,Code,Symbol,ThousandsSeparator,DecimalSeparator,SymbolOnLeft,SpaceBetweenAmountAndSymbol,RoundingCoefficient,DecimalDigits
0,USD,$,",",.,True,False,0,2
1,USD,$,",",.,True,False,0,2


In [ ]:
# I now have everything unpacked, but need to find a way to combine the dataframes using the original indexing...